# Creating nifti betas for TDT

In [ ]:
import numpy as np
import nibabel as nib
from nilearn import plotting, image
import pandas as pd
import os
import glob
from os.path import join, exists, split
from scipy.stats import pearsonr
import re

outdir = '/Users/danieljanini/Documents/Thesis/miniblock/Outputs/'
homedir = split(os.getcwd())[0]
datadir = '/Users/danieljanini/Documents/Thesis/miniblock/'
presdir = join(homedir, 'Behavior', 'designmats')
subs = ["01", "02", "03","04", "05","06", "07", "08", "10", "11", "12", "13", "14", "15", "17", "18", "19"]
smooths = ["sm_2_vox", "unsmoothed"]
runtypes = ['miniblock', "sus", "er"]

# Define affine (identity or use a real affine if you have one)
target_affine = np.array([
    [2., 0., 0., -76.],
    [0., 2., 0., -112.],
    [0., 0., 2., -76.],
    [0., 0., 0., 1.]
])
target_shape = (77, 95, 82)
affine = target_affine

for sub in subs: 
    for smoothing in smooths: 
        for runtype in runtypes: 

            results_glmsingle = dict()
            results_glmsingle['typed'] = np.load(join(outdir,f'sub-{sub}',f'{smoothing}_sub-{sub}_{runtype}/TYPED_FITHRF_GLMDENOISE_RR.npy'), allow_pickle=True).item()
            betas = results_glmsingle['typed']['betasmd']



            # Output folder for NIfTIs
            save_dir = join(outdir, f'sub-{sub}', f'{smoothing}_sub-{sub}_{runtype}', 'nifti_betas')
            os.makedirs(save_dir, exist_ok=True)

            # Define affine (identity or use a real affine if you have one)
            affine = target_affine

            # Loop over trials and save each one
            n_trials = betas.shape[-1]
            for i in range(n_trials):
                beta_data = betas[..., i]  # Take 3D volume for trial i
                nii = nib.Nifti1Image(beta_data, affine)
                nib.save(nii, join(save_dir, f'beta_{i+1:04d}.nii'))
    


In [ ]:
# Load and process design matrix
pattern = presdir + f'/P0{sub}_ConditionRich_Run*_{runtype}.csv'
matches = glob.glob(pattern)
matches.sort()
design = []
for i in range(len(matches)):
    designMat = pd.read_csv(matches[i], header=None)
    num = re.search(r'Run_(\d+)', matches[i])
    runNum = int(num.group(1))
    # Adjust runNum for interspersed localizer runs
    if (runNum > 3) & (runNum < 7) & (sub != '01'): 
        runNum += 1
    elif (runNum >= 7):
        runNum += 2
    elif (sub == '01') & (runNum > 4) & (runNum < 7):
        runNum += 1
    design.append(designMat)

all_design = np.vstack((design[0], design[1], design[2]))
condition_mask = all_design.sum(axis=1) > 0
condition_vector = np.argmax(all_design[condition_mask], axis=1)
n_conditions = 40
max_reps = 6

repindices = np.full((max_reps, n_conditions), np.nan)
for p in range(n_conditions):  
    inds = np.where(condition_vector == p)[0]  
    repindices[:len(inds), p] = inds  